In [2]:
import os
import cv2
import numpy as np

In [3]:
# Directories for the original and augmented images
input_dir_yes = 'dataset/YES/'
input_dir_no = 'dataset/NO/'
augmented_dir_yes = 'dataset/Augmented/YES/'
augmented_dir_no = 'dataset/Augmented/NO/'

# Ensure the augmented directories exist
os.makedirs(augmented_dir_yes, exist_ok=True)
os.makedirs(augmented_dir_no, exist_ok=True)


In [4]:
# Function to apply augmentations
def augment_image(image, num_augmentations=100):
    augmented_images = []
    
    for _ in range(num_augmentations):
        aug_img = image.copy()
        
        # Random rotation
        angle = np.random.choice([0, 90, 180, 270])
        if angle != 0:
            aug_img = cv2.rotate(aug_img, cv2.ROTATE_90_CLOCKWISE if angle == 90 else cv2.ROTATE_180 if angle == 180 else cv2.ROTATE_90_COUNTERCLOCKWISE)
        
        # Random horizontal flip
        if np.random.rand() > 0.5:
            aug_img = cv2.flip(aug_img, 1)
        
        # Random scaling (zoom in/out)
        scale_factor = np.random.uniform(0.8, 1.2)
        h, w = aug_img.shape[:2]
        aug_img = cv2.resize(aug_img, (int(w * scale_factor), int(h * scale_factor)))
        aug_img = cv2.resize(aug_img, (w, h))  # Resize back to original size
        
        # Add random noise
        noise = np.random.randint(0, 50, (aug_img.shape[0], aug_img.shape[1]), dtype='uint8')
        aug_img = cv2.add(aug_img, noise)
        
        # Random brightness/contrast adjustment
        alpha = np.random.uniform(0.8, 1.2)  # Contrast control
        beta = np.random.randint(-30, 30)    # Brightness control
        aug_img = cv2.convertScaleAbs(aug_img, alpha=alpha, beta=beta)
        
        augmented_images.append(aug_img)
    
    return augmented_images

In [5]:
# Function to process and augment images in a directory
def process_and_augment_images(input_dir, augmented_dir, num_augmentations_per_image=100):
    for heatmap_file in os.listdir(input_dir):
        if heatmap_file.endswith('.png'):
            # Load the original heatmap
            heatmap = cv2.imread(os.path.join(input_dir, heatmap_file), cv2.IMREAD_GRAYSCALE)
            
            # Save the original heatmap in the augmented directory
            cv2.imwrite(os.path.join(augmented_dir, heatmap_file), heatmap)
            
            # Augment the image
            augmented_images = augment_image(heatmap, num_augmentations=num_augmentations_per_image)
            
            # Save the augmented images
            for idx, aug_img in enumerate(augmented_images):
                aug_filename = f'{os.path.splitext(heatmap_file)[0]}_aug{idx}.png'
                cv2.imwrite(os.path.join(augmented_dir, aug_filename), aug_img)

In [9]:
# Function to get the list of image files in a directory
def get_image_list(input_dir):
    return [f for f in os.listdir(input_dir) if f.endswith('.png')]

In [ ]:
# Augment 'YES' and 'NO' heatmaps
process_and_augment_images(input_dir_yes, augmented_dir_yes, num_augmentations_per_image=100)
process_and_augment_images(input_dir_no, augmented_dir_no, num_augmentations_per_image=100)

print('Data augmentation complete!')

Processing images:   0%|          | 0/10 [00:00<?, ?it/s]


TypeError: augment_image() takes from 1 to 2 positional arguments but 3 were given